# Assignment 4: Shengdi Lin

### 3 Practical Applications of Constraint Satisfaction Problems (CSP):
1. Natural Language Processing (NLP): In various NLP tasks such as syntactic parsing, semantic analysis, and language generation, CSPs can be used to enforce linguistic constraints. For example, in syntactic parsing, CSPs can help in constructing parse trees that adhere to grammar rules and word dependencies. In semantic analysis, CSPs can assist in resolving ambiguities and inconsistencies in language understanding by enforcing constraints based on semantic rules and contextual information. This allows for the return of results, such as in ChatGPT, that make actual sense because they follow the convention of our semantics. 

2. Resource Allocation: CSPs can be employed to model these allocation problems and find solutions that maximize resource utilization while ensuring timely results, staff satisfaction, and operational efficiency. One example is healthcare, where resource allocation is seen in tasks such as assigning hospital beds, scheduling surgeries, or allocating medical staff. These involve dealing with multiple constraints like resource availability, patient urgency, and medical staff preferences.

3. Timetabling and Course Scheduling: Educational institutions face challenges in creating timetables and schedules for classes, exams, and other academic activities that satisfy various constraints like room availability, faculty preferences, course prerequisites, and student preferences. CSPs can be applied to generate optimal timetables that minimize conflicts and maximize resource utilization while meeting all constraints. This can lead to more efficient use of resources and improved academic outcomes for students.


## Why is Sudoku a Constraint Satisfaction Problem?
Sudoku has two sets of main rules. All boxes in a 3x3 grid must have one instance of each number 1-9. All rows and columns must contain the numbers 1-9. Following these two rules, one can attempt to solve for the goal state, where the entire grid is filled and the above rules are satisfied. There is only one unique goal state. There are a set number of total scenarios that are possible, making it a discrete environment. The sudoku also does not change, making its environment static. The sudoka is entirely observable, meaning no information is withdrawn besides what we are solving for, and since the next state of the environment is completely determined by the current state and the actions of the agent (the solver), then the environment is also deterministic. **By setting these rules and transcribing it into code, we can approach solving sudokus as a constraint satisfaction problem.**

These rules allow for when one state gets updated, you can get to the solution by getting to state to state. There are two main rules:

(1) If a square has only one possible value, then eliminate that value from the square's peers (rows, cols, squares)
(2) If a unit has only one possible place for a value, then put the value there.

As an example of strategy (1) if we assign 7 to A1 (the top left spot), we see that A1 has only one value, and thus the 7 can be removed from its peer A2 (and all other peers). As a separate example of strategy (2), if it turns out that none of A2 through A9 has a 3 as a possible value, then the 3 must belong in A1. This process is called constraint propagation, because we use set constraints to move through different states with the arrival of more and more information building on each other. 




In [2]:
def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
#For each unit, tie it to all peers it has categorized in three relationships (rows, cols, squares) and have each relationship type as one array
units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
#For each unit, tie it to all its peers in EVERY relationship
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

Create an initial test to see if everything has been structured correctly. 

In [3]:
#THIS WAS CREATED TO CREATE A TEST TO ENSURE THAT THE STRUCTURE OF THE SUDOKU IS SET UP CORRECTLY 
#CHECKS THE CELL C2 SPECIFICALLY FOR UNITS AND PEERS  
def test():
    "A set of unit tests."
    assert len(squares) == 81
    assert len(unitlist) == 27
    assert all(len(units[s]) == 3 for s in squares)
    assert all(len(peers[s]) == 20 for s in squares)
    assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                           ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                           ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
    assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                               'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                               'A1', 'A3', 'B1', 'B3'])
    print ('All tests pass.')

test()

All tests pass.


Our values will be a dict with squares as keys. The value of each key will be the possible digits for that square: a single digit if it was given as part of the puzzle definition or if we have figured out what it must be, and a collection of several digits if we are still uncertain. We have already defined that each row is assigned a letter from "A-I" and each column is assigned a number "1-9". Now let's figure out how to read in our data appropriately. 

We will need two representations of the sudoku grid: First, a textual format used to specify the initial state of a puzzle; we will reserve the name **GRID** for this. 

Second, an internal representation of any state of a puzzle, partially solved or complete; this we will call a **VALUES** collection because it will give all the remaining possible values for each square. 

For the textual format (grid) we'll allow a string of characters with 1-9 indicating a digit, and a 0 or period specifying an empty square. All other characters are ignored (including spaces, newlines, dashes, and bars) when it is initially read in. 

Let' start by creating a method that allows us to create our VALUES collection from a GRID of numbers. Then, we will create an additional method to create a GRID of numbers from an user-inputted sudoku puzzle. 

In [14]:
def parse_grid(grid):
    """Take the grid given and return it in an appropriate fashion
    for displaying."""
    values = dict((s, 0) for s in squares)
    place = 0
    for s in squares:
        values[s] = grid[place]
        place += 1
    return values

def solve_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

Now, we need to create code to eliminate possible scenarios that comes with new information. This new information is "sent" to the new state through the assign method. 

In [6]:
def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    else:
        values[s] = values[s].replace(d,'')
        ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
        if len(values[s]) == 0:
            return False ## Contradiction: removed last valu
        elif len(values[s]) == 1:
            d2 = values[s]
            if not all(eliminate(values, s2, d2) for s2 in peers[s]):
                return False
        ## (2) If a unit u is reduced to only one place for a value d, then put it there.
        else:
            for u in units[s]:
                dplaces = [s for s in u if d in values[s]]
            if len(dplaces) == 0:
                return False ## Contradiction: no place for this value
            elif len(dplaces) == 1:
                # d can only be in one place in unit; assign it there
                if not assign(values, dplaces[0], d):
                    return False
    return values

Now that we have our assign and eliminate methods that look for and eliminiate scenarios that violate our rules, we should create a way to display the sudoku puzzle. 

In [11]:
def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print (''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': 
            print(line)
    print


Let us load a sample grid and display it from the kaggle dataset, found here: https://www.kaggle.com/datasets/rohanrao/sudoku?select=sudoku.csv

In [17]:
grid1 = "048301560360008090910670003020000935509010200670020010004002107090100008150834029"
display(parse_grid(grid1))


0 4 8 |3 0 1 |5 6 0 
3 6 0 |0 0 8 |0 9 0 
9 1 0 |6 7 0 |0 0 3 
------+------+------
0 2 0 |0 0 0 |9 3 5 
5 0 9 |0 1 0 |2 0 0 
6 7 0 |0 2 0 |0 1 0 
------+------+------
0 0 4 |0 0 2 |1 0 7 
0 9 0 |1 0 0 |0 0 8 
1 5 0 |8 3 4 |0 2 9 


Display the solved grid:

In [18]:
display(solve_grid(grid1))

7 4 8 |3 9 1 |5 6 2 
3 6 5 |2 4 8 |7 9 1 
9 1 2 |6 7 5 |4 8 3 
------+------+------
4 2 1 |7 8 6 |9 3 5 
5 8 9 |4 1 3 |2 7 6 
6 7 3 |5 2 9 |8 1 4 
------+------+------
8 3 4 |9 6 2 |1 5 7 
2 9 6 |1 5 7 |3 4 8 
1 5 7 |8 3 4 |6 2 9 


However, there are certain instances where the two above rules **DON'T** get us completely to the end solution. We will show an example below:

In [19]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
display(parse_grid(grid2))

4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 


Let us try to solve it with our method above.

In [20]:
display(solve_grid(grid2))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


Evidently, there are still many possible states for this sudoku puzzle. Even though there is one unique solution, **WE NEED TO TAKE THE EXTRA STEP OF CONSIDERING POSSIBLE SCENARIOS AND DELETING THEM IF THEY DO NOT WORK**. To do so we have a few things to consider. 

First, that whenever we make a decision for the possible states, we eliminate a ton of possibilities. In square H7 of our grid2, we have two possibilities, 6 and 9. We can try 9 and quickly see that there is a contradiction. That means we've eliminated not just one possibility, but fully half of the possible choices that had included 9 in spot H7. 

Second, how we search through these possible scenarios. First make sure we haven't already found a solution or a contradiction using the code we already have. If we do not, we choose one unfilled square and consider all its possible values. One at a time, we try assigning the square's possible values and searching from the resulting position. In other words, we search for a value $d$ such that we can successfully search for a solution from the result of assigning $d$ to that square. If the search leads to an failed position, go back and consider another value of $d$. This is a **depth-first search** because we are recursively considering all possibilities for assigning $d$ to a square before we consider a different value of $d$ for that square.

Third, we create this as a **backtracking search** by keeping track of each change to values and undo the changes when we hit a dead end. By doing so, we move back up to the highest level where a possible candidate exists and continue going through the possibilities. 

In [32]:
# Take it to the next step and complete solving the sudoku grid given possible states
def solveComplete(grid): 
    toGrid = solve_grid(grid)
    return search(toGrid)

# Go through possibilities, first with squares that have least possible values 
# We update values so that if one scenario is not possible, all other possible values for other peers track that as no longer a possibility
def search(values):
    "Using depth-first search and propagation, try all possible values."
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in squares): 
        return values ## Solved!
    ## Chose the unfilled square s with the fewest possibilities
    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    return some(search(assign(values.copy(), s, d)) 
		for d in values[s])

# What we use to backtrack -- as stated in above, if it fails we mark as failed (dead end)
# and go to another possibility in the sequence of possible numbers for that square s
def some(seq):
    "Return some element of seq that is true."
    for e in seq:
        if e: return e
    return False

In [33]:
display(solveComplete(grid2))

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 


So, now we can see that using back-tracking and constraint propropogation, we can create a sudoku solver that considers all possible scenarios to arrive at the one goal state. We will use more sudokus from kaggle to see how it performs. First, let's import the dataset from kaggle and show the first entry and its solution: https://www.kaggle.com/datasets/rohanrao/sudoku?select=sudoku.csv

ENSURE YOU HAVE THIS KAGGLE DATASET DOWNLOADED IN THE SAME DIRECTORY AS DETAILED IN THE README

In [38]:
# import appropriate libraries
import random as ran
import csv
import time
import numpy as np

In [44]:
# Initialize empty arrays to store data from CSV
arr1 = []
arr2 = []

# Open the CSV file for reading
with open('sudoku.csv', newline='') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)
    
    # Iterate over each row in the CSV file
    for row in reader:
        # Append the first column to arr1 and the second column to arr2
        arr1.append(row[0])
        arr2.append(row[1])

# delete headers
del arr1[0]
del arr2[0]


# Now arr1 and arr2 contain the data from column 1 and column 2 respectively
print("arr1 first element:", arr1[0])
print("arr2 first element:", arr2[0])

arr1 first element: 070000043040009610800634900094052000358460020000800530080070091902100005007040802
arr2 first element: 679518243543729618821634957794352186358461729216897534485276391962183475137945862


In [68]:
# Go through trials with our model and display results for each trial as well as a summary of performance. 
def trials(numTrials): 
    # take n random samples and see how it did as well as how long it took to solve for each one
    timeTaken = []
    numRight = 0
    for i in range(numTrials):
        rand = ran.randint(0, len(arr1))
        given = parse_grid(arr1[rand])
        solu = parse_grid(arr2[rand])
        start_time = time.time() # time taken also takes into account writing the solution out to object solved
        solved = solveComplete(arr1[rand])
        end_time = time.time()

        duration = end_time - start_time
        # get all the timed durations 
        timeTaken.append(duration)

        # print the sudokus and times
        print("Sudoku #", i+1, "(given): ")
        display(given)
        print("Sudoku #", i+1, "(solved):")
        display(solved)
        # if they match, add to number of correct givens.
        if solved == solu:
            numRight = numRight + 1
        print("Time taken: ", duration)
        
    #get average time taken
    average = np.mean(timeTaken)
    # Round the average to 6 decimal points
    rounded_average = round(average, 6)
    print()
    print("SUMMARY: ")
    print("Out of", numTrials,"total sudokus, our model had", numRight, "correct solves")
    print("The model took an average of", rounded_average, "seconds for each iteration of solving")

# Create a summary ONLY function 
def massTrials(numTrials): 
    # take n random samples and return a summary ONLY of results
    timeTaken = []
    numRight = 0
    for i in range(numTrials):
        rand = ran.randint(0, len(arr1))
        given = parse_grid(arr1[rand])
        solu = parse_grid(arr2[rand])
        start_time = time.time() # time taken also takes into account writing the solution out to object solved
        solved = solveComplete(arr1[rand])
        end_time = time.time()

        duration = end_time - start_time
        # get all the timed durations 
        timeTaken.append(duration)

        # if they match, add to number of correct givens.
        if solved == solu:
            numRight = numRight + 1
        
    #get average time taken
    average = np.mean(timeTaken)
    # Round the average to 6 decimal points
    rounded_average = round(average, 6)
    print()
    print("SUMMARY: ")
    print("Out of", numTrials,"total sudokus, our model had", numRight, "correct solves")
    print("The model took an average of", rounded_average, "seconds for each iteration of solving")

In [73]:
ran.seed(360)
trials(5)

Sudoku # 1 (given): 
0 0 0 |0 4 8 |0 0 9 
0 7 3 |9 1 2 |8 0 0 
0 9 0 |6 7 5 |0 0 0 
------+------+------
7 0 9 |0 3 1 |0 4 0 
0 1 4 |0 0 6 |0 9 0 
6 5 2 |0 8 9 |7 3 0 
------+------+------
0 2 0 |1 0 3 |4 5 0 
5 0 1 |0 0 0 |0 0 0 
9 0 7 |0 0 0 |0 1 6 
Sudoku # 1 (solved):
2 6 5 |3 4 8 |1 7 9 
4 7 3 |9 1 2 |8 6 5 
1 9 8 |6 7 5 |3 2 4 
------+------+------
7 8 9 |5 3 1 |6 4 2 
3 1 4 |7 2 6 |5 9 8 
6 5 2 |4 8 9 |7 3 1 
------+------+------
8 2 6 |1 9 3 |4 5 7 
5 4 1 |2 6 7 |9 8 3 
9 3 7 |8 5 4 |2 1 6 
Time taken:  0.0032401084899902344
Sudoku # 2 (given): 
4 0 1 |2 9 0 |8 0 6 
3 6 9 |7 0 8 |0 0 0 
2 0 0 |0 0 0 |9 0 0 
------+------+------
0 0 6 |0 0 2 |1 5 0 
5 3 0 |1 6 0 |0 8 0 
7 1 0 |0 5 0 |2 0 0 
------+------+------
0 0 8 |0 1 7 |0 0 0 
0 5 3 |9 8 4 |0 0 2 
1 4 7 |0 2 0 |0 9 8 
Sudoku # 2 (solved):
4 7 1 |2 9 5 |8 3 6 
3 6 9 |7 4 8 |5 2 1 
2 8 5 |6 3 1 |9 7 4 
------+------+------
8 9 6 |4 7 2 |1 5 3 
5 3 2 |1 6 9 |4 8 7 
7 1 4 |8 5 3 |2 6 9 
------+------+------
9 2 8 |3 1 7 |6 4 5 

Now, we see the results of 5 trials that saw our model get the correct solution at an average speed of roughly 0.0035 seconds. Let's see how it did with larger batches.

In [74]:
massTrials(1000)
massTrials(5000)
massTrials(10000)


SUMMARY: 
Out of 1000 total sudokus, our model had 1000 correct solves
The model took an average of 0.002407 seconds for each iteration of solving

SUMMARY: 
Out of 5000 total sudokus, our model had 5000 correct solves
The model took an average of 0.002503 seconds for each iteration of solving

SUMMARY: 
Out of 10000 total sudokus, our model had 10000 correct solves
The model took an average of 0.002489 seconds for each iteration of solving


# Conclusion

Our AI agent did well on using backtracking to help in its solving of sudokus using a constraint propogation approach. 